In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pickle
with open('pivot4years.pkl','rb') as pkldfile :
    pivotByYear=pickle.load(pkldfile)

In [3]:
pivotByYear.head()

AttributeError: 'dict' object has no attribute 'head'

In [4]:
for i in pivotByYear.keys() : print(pivotByYear[i].shape) 

(1845, 1)
(1839, 1)
(1837, 1)
(1840, 1)
(1837, 1)
(1840, 1)
(1845, 1)


In [5]:
import geopandas as gpd
smallcounties = gpd.read_file('cb_2018_us_county_500k/cb_2018_us_county_500k.shp')


In [6]:
state_dict={ '25':'Massachusetts', '09':'Connecticut' , '23':'Maine', '33':'New Hampshire', '50':'Vermont' ,'44':'Rhode Island'  }
necounties=smallcounties[smallcounties['STATEFP'].isin(state_dict.keys())].copy()

In [7]:
for i in pivotByYear.keys():
    pivotByYear[i].index=pivotByYear[i].index.str.title()

In [8]:
geoOPyear=dict()
for i in pivotByYear.keys():
    geoOPyear[i]=necounties.merge(pivotByYear[i],left_on='NAME',right_on='BUYER_COUNTY')
    geoOPyear[i].drop(columns=['ALAND','AWATER'])

In [9]:
for i in geoOPyear.keys() : print(geoOPyear[i].shape) 

(67, 11)
(67, 11)
(67, 11)
(67, 11)
(67, 11)
(67, 11)
(67, 11)


In [10]:
for i in pivotByYear.keys() : print(pivotByYear[i].isna().sum()) 

TRANSACTION_DATE
2006    0
dtype: int64
TRANSACTION_DATE
2007    0
dtype: int64
TRANSACTION_DATE
2008    0
dtype: int64
TRANSACTION_DATE
2009    0
dtype: int64
TRANSACTION_DATE
2010    0
dtype: int64
TRANSACTION_DATE
2011    0
dtype: int64
TRANSACTION_DATE
2012    0
dtype: int64


In [11]:
for i in geoOPyear.keys():
    geoOPyear[i]['inMillions']=geoOPyear[i][int(i)].divide(10**6)

In [12]:
death_dtypes={'Year':int,'County':str,'Notes':str,'County Code':int,'Year Code':int , 'Deaths':pd.Int32Dtype(),'Population':int,'Crude Rate':float }
deaths=pd.read_csv('MCDNewEngland20062012.tsv',sep='\t',index_col=['County','Year'],dtype=death_dtypes, na_values=['Suppressed','Unreliable'])

In [13]:
deathspivot=deaths['Deaths'].unstack()

In [14]:
deathspivot.head()

Year,2006,2007,2008,2009,2010,2011,2012
County,,,,,,,
"Addison County, VT",NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Androscoggin County, ME",NaN,NaN,NaN,10,11,NaN,NaN
"Aroostook County, ME",NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Barnstable County, MA",22,29,22,20,20,16,19
"Belknap County, NH",NaN,12,10,NaN,NaN,NaN,NaN


In [15]:
deathspivot['CName']=deathspivot.index.str.split().str.get(0)

In [16]:
deathspivot.head()

Year,2006,2007,2008,2009,2010,2011,2012,CName
County,,,,,,,,
"Addison County, VT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Addison
"Androscoggin County, ME",NaN,NaN,NaN,10,11,NaN,NaN,Androscoggin
"Aroostook County, ME",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aroostook
"Barnstable County, MA",22,29,22,20,20,16,19,Barnstable
"Belknap County, NH",NaN,12,10,NaN,NaN,NaN,NaN,Belknap


In [20]:
geoOPyear.keys()

dict_keys(['2006', '2007', '2008', '2009', '2010', '2011', '2012'])

In [25]:
geoOPyear['2006'].head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,2006,inMillions
0,09,009,00212798,0500000US09009,09009,New Haven,06,1565688367,667081242,"MULTIPOLYGON (((-72.76143 41.24233, -72.75973 ...",24486606.0,24.486606
1,23,013,00581292,0500000US23013,23013,Knox,06,945684684,2017116243,"MULTIPOLYGON (((-68.55926 44.04502, -68.55504 ...",34936280.0,34.936280
2,23,015,00581293,0500000US23015,23015,Lincoln,06,1180688991,631299424,"MULTIPOLYGON (((-69.32357 43.75899, -69.32114 ...",19294160.0,19.294160
3,25,001,00606927,0500000US25001,25001,Barnstable,06,1021048430,2360287543,"POLYGON ((-70.68698 41.52949, -70.68327 41.532...",6526795.0,6.526795
4,25,013,00606933,0500000US25013,25013,Hampden,06,1597996708,44417070,"POLYGON ((-73.07484 42.10615, -73.07305 42.106...",13992080.0,13.992080


In [31]:
deathspivot.columns

Index([2006, 2007, 2008, 2009, 2010, 2011, 2012, 'CName'], dtype='object', name='Year')

In [35]:
deathspivot[[2006,'CName']]

Year,2006,CName
County,,
"Addison County, VT",NaN,Addison
"Androscoggin County, ME",NaN,Androscoggin
"Aroostook County, ME",NaN,Aroostook
"Barnstable County, MA",22,Barnstable
"Belknap County, NH",NaN,Belknap
...,...,...
"Windham County, CT",10,Windham
"Windham County, VT",NaN,Windham
"Windsor County, VT",NaN,Windsor


In [57]:
for i in deathspivot.index :
    

Addison County, VT
Androscoggin County, ME
Aroostook County, ME
Barnstable County, MA
Belknap County, NH
Bennington County, VT
Berkshire County, MA
Bristol County, MA
Bristol County, RI
Caledonia County, VT
Carroll County, NH
Cheshire County, NH
Chittenden County, VT
Coos County, NH
Cumberland County, ME
Dukes County, MA
Essex County, MA
Essex County, VT
Fairfield County, CT
Franklin County, MA
Franklin County, ME
Franklin County, VT
Grafton County, NH
Grand Isle County, VT
Hampden County, MA
Hampshire County, MA
Hancock County, ME
Hartford County, CT
Hillsborough County, NH
Kennebec County, ME
Kent County, RI
Knox County, ME
Lamoille County, VT
Lincoln County, ME
Litchfield County, CT
Merrimack County, NH
Middlesex County, CT
Middlesex County, MA
Nantucket County, MA
New Haven County, CT
New London County, CT
Newport County, RI
Norfolk County, MA
Orange County, VT
Orleans County, VT
Oxford County, ME
Penobscot County, ME
Piscataquis County, ME
Plymouth County, MA
Providence County, RI

In [36]:
geoDeath=dict()
for i in geoOPyear.keys():
    temp=deathspivot[[int(i),'CName']]
    geoDeath[i]=geoOPyear[i].merge(temp,left_on='NAME',right_on='CName')
    geoDeath[i].drop(columns=['ALAND','AWATER'])

In [44]:
geoDeath['2006'].isna().sum()

STATEFP        0
COUNTYFP       0
COUNTYNS       0
AFFGEOID       0
GEOID          0
NAME           0
LSAD           0
ALAND          0
AWATER         0
geometry       0
2006_x         0
inMillions     0
2006_y        57
CName          0
dtype: int64

In [45]:
geoDeath['2006'].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 84 entries, 0 to 83
Data columns (total 14 columns):
STATEFP       84 non-null object
COUNTYFP      84 non-null object
COUNTYNS      84 non-null object
AFFGEOID      84 non-null object
GEOID         84 non-null object
NAME          84 non-null object
LSAD          84 non-null object
ALAND         84 non-null int64
AWATER        84 non-null int64
geometry      84 non-null geometry
2006_x        84 non-null float32
inMillions    84 non-null float64
2006_y        27 non-null Int32
CName         84 non-null object
dtypes: Int32(1), float32(1), float64(1), geometry(1), int64(2), object(8)
memory usage: 9.3+ KB


In [47]:
geoOPyear['2006'].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 67 entries, 0 to 66
Data columns (total 12 columns):
STATEFP       67 non-null object
COUNTYFP      67 non-null object
COUNTYNS      67 non-null object
AFFGEOID      67 non-null object
GEOID         67 non-null object
NAME          67 non-null object
LSAD          67 non-null object
ALAND         67 non-null int64
AWATER        67 non-null int64
geometry      67 non-null geometry
2006          67 non-null float32
inMillions    67 non-null float64
dtypes: float32(1), float64(1), geometry(1), int64(2), object(7)
memory usage: 6.5+ KB


In [48]:
deathspivot.shape

(67, 8)

In [50]:
list(geoDeath['2006']['NAME'])

['Knox',
 'Lincoln',
 'Barnstable',
 'Hampden',
 'Suffolk',
 'Dukes',
 'Middlesex',
 'Middlesex',
 'Middlesex',
 'Middlesex',
 'Sagadahoc',
 'York',
 'Coos',
 'Rockingham',
 'Strafford',
 'Orange',
 'Windsor',
 'Franklin',
 'Franklin',
 'Franklin',
 'Franklin',
 'Franklin',
 'Franklin',
 'Franklin',
 'Franklin',
 'Franklin',
 'Fairfield',
 'Aroostook',
 'Somerset',
 'Bristol',
 'Bristol',
 'Bristol',
 'Bristol',
 'Washington',
 'Washington',
 'Washington',
 'Washington',
 'Washington',
 'Washington',
 'Washington',
 'Washington',
 'Washington',
 'Providence',
 'Waldo',
 'Litchfield',
 'Tolland',
 'Kennebec',
 'Essex',
 'Essex',
 'Essex',
 'Essex',
 'Berkshire',
 'Norfolk',
 'Windham',
 'Windham',
 'Windham',
 'Windham',
 'Penobscot',
 'Orleans',
 'Carroll',
 'Piscataquis',
 'Merrimack',
 'Bennington',
 'Nantucket',
 'Belknap',
 'Kent',
 'Hartford',
 'Chittenden',
 'Cheshire',
 'Plymouth',
 'Hancock',
 'Newport',
 'Lamoille',
 'Cumberland',
 'Hillsborough',
 'Addison',
 'Sullivan',
 'Ru

In [53]:
list(deathspivot.index)

['Addison County, VT',
 'Androscoggin County, ME',
 'Aroostook County, ME',
 'Barnstable County, MA',
 'Belknap County, NH',
 'Bennington County, VT',
 'Berkshire County, MA',
 'Bristol County, MA',
 'Bristol County, RI',
 'Caledonia County, VT',
 'Carroll County, NH',
 'Cheshire County, NH',
 'Chittenden County, VT',
 'Coos County, NH',
 'Cumberland County, ME',
 'Dukes County, MA',
 'Essex County, MA',
 'Essex County, VT',
 'Fairfield County, CT',
 'Franklin County, MA',
 'Franklin County, ME',
 'Franklin County, VT',
 'Grafton County, NH',
 'Grand Isle County, VT',
 'Hampden County, MA',
 'Hampshire County, MA',
 'Hancock County, ME',
 'Hartford County, CT',
 'Hillsborough County, NH',
 'Kennebec County, ME',
 'Kent County, RI',
 'Knox County, ME',
 'Lamoille County, VT',
 'Lincoln County, ME',
 'Litchfield County, CT',
 'Merrimack County, NH',
 'Middlesex County, CT',
 'Middlesex County, MA',
 'Nantucket County, MA',
 'New Haven County, CT',
 'New London County, CT',
 'Newport Coun

In [54]:
deaths.shape

(469, 6)

In [55]:
deathspivot.shape

(67, 8)

In [56]:
deathspivot.head()

Year,2006,2007,2008,2009,2010,2011,2012,CName
County,,,,,,,,
"Addison County, VT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Addison
"Androscoggin County, ME",NaN,NaN,NaN,10,11,NaN,NaN,Androscoggin
"Aroostook County, ME",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aroostook
"Barnstable County, MA",22,29,22,20,20,16,19,Barnstable
"Belknap County, NH",NaN,12,10,NaN,NaN,NaN,NaN,Belknap
